# JSONファイルからRAG用のベクトルDBを作る

LangChain を使って RAG を試してみた #AI - Qiita
> https://qiita.com/tinymouse/items/4d359674f6b2494bb22d

LLMアプリケーション開発のためのLangChain 後編⑤ 外部ドキュメントのロード、分割及び保存 - qiita
> https://qiita.com/utanesuke/items/6efc03eca94f7de3b9cd#json-%E3%83%AD%E3%83%BC%E3%83%80%E3%83%BC


## install

In [ ]:
!pip install unsloth langchain langchain_community langchain-huggingface sentence-transformers transformers accelerate chromadb

In [ ]:
!pip install 

## imports

In [81]:
## main models
# need to import unsloth 1st
from unsloth import FastLanguageModel
import transformers
from sentence_transformers import SentenceTransformer
import torch

## langchain (need to make RAG)
from langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings
from langchain_community.document_loaders import JSONLoader
from langchain_community.vectorstores import Chroma

import json
from pathlib import Path
import pprint


## prepare tokenizer & model

In [66]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    max_seq_length = 40960,
    load_in_4bit = True,            # 4bit uses much less memory
    load_in_8bit = False,           # A bit more accurate, uses 2x memory
)

==((====))==  Unsloth 2025.9.11: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    NVIDIA RTX A6000. Num GPUs = 1. Max memory: 47.431 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## prepare pipeline

In [6]:
pipe = transformers.pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    dtype=torch.float16
)

llm = HuggingFacePipeline(
    pipeline = pipe
)

Device set to use cuda:0


## load json datas

In [69]:
# # need to avoid unicode escape & marge instruction, output
# loader = JSONLoader(
#     file_path="./jvn_results_merged.json",
# #    jq_schema=".[]",
#     jq_schema=".[] | .instruction, .output",
#     text_content=False
# )
# docs = loader.load()

In [74]:
# ひとまずこれで試してみる
file_path = "./jvn_results_merged.json"
docs = json.loads(Path(file_path).read_text())

In [79]:
pprint.pprint(docs[1])

{'instruction': 'SQL インジェクションの脆弱性の例を教えて',
 'output': 'carmelogarcia の Simple Leave Manager In PHP With Source Code における '
           'SQL インジェクションの脆弱性が発表された。A flaw has been found in code-projects '
           'Simple Leave Manager 1.0. This vulnerability affects unknown code '
           'of the file /user.php. This manipulation of the argument table '
           'causes sql injection. Remote exploitation of the attack is '
           'possible. The exploit has been published and may be used. '
           'この脆弱性はcarmelogarcia\n'
           'Simple Leave Manager In PHP With Source Code 1.0に影響を及ぼす。'}


## prepare embedding model

In [84]:
embeddings = HuggingFaceEmbeddings(
    # need searching better performance & can download
    model_name = "Qwen/Qwen3-Embedding-8B"
)

KeyboardInterrupt: 

## save DB

In [85]:
vectorstore = Chroma.from_documents(
    documents = docs,
    embeddings = embeddings
)

NameError: name 'embeddings' is not defined

## search DB

## add context for prompt